In [48]:
UB=1000
LB=-1000
prob=0.2
scenario=[0.6,0.8,1,1.2,1.4]
P=[[] for i in range(len(scenario))]
R=[[] for i in range(len(scenario))]

f=20
I=10
J=5
c = [[1.3, 0.7, 0.1, 0.3, 0.0, 0.9, 1.0, 0.1, 0.3, 0.1],
     [0.0, 0.2, 0.3, 0.2, 0.9, 0.4, 0.2, 0.8, 0.1, 0.6],
     [0.4, 0.3, 1.5, 0.0, 0.5, 0.5, 0.2, 0.4, 1.0, 0.4],
     [0.5, 0.5, 0.2, 0.1, 0.7, 0.8, 0.8, 0.2, 0.0, 1.1],
     [0.2, 0.3, 0.1, 0.7, 0.2, 0.9, 0.4, 0.6, 0.3, 1.4]]
C = 100
f = 20
d = [15, 23, 26, 12, 31, 19, 25, 17, 21, 14]

In [49]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import time
S=5
results={}

In [50]:
tic=time.perf_counter()
for g in (0.1,0.5,1,10,20,100,1000):
    simple=gp.Model()
    x=simple.addVars(J ,vtype=GRB.BINARY ,name='x')
    y=simple.addVars(I,J, vtype=GRB.CONTINUOUS, name='y')
    zplus=simple.addVars(I,S, vtype=GRB.CONTINUOUS, name='zplus')
    zminus=simple.addVars(I,S, vtype=GRB.CONTINUOUS, name='zminus')

    obj=0
    for s in range(S):
        obj1=0
        for i in range (I):
            obj1+=g*(zminus[i,s]+zplus[i,s])

        obj+=prob*obj1

    simple.setObjective(quicksum(f*x[j] for j in range(J))+quicksum(quicksum(c[j][i]*y[i,j] for i in range(I)) for j in range(J))\
                       +obj, GRB.MINIMIZE)


    for i in range (I):
        for s in range(S):
            simple.addConstr(quicksum(y[i,j]  for j in range(J))+ zplus[i,s]-zminus[i,s]==d[i]*scenario[s])

    for j in range (J):
        simple.addConstr(quicksum(y[i,j] for i in range(I))<=C*x[j])

    simple.optimize()
    xs=[]
    for i in range(5):
        xs.append(x[i].x)
    
    results[g]=int(simple.objVal), xs
    

x=xs
toc=time.perf_counter()
print('runtime :', toc-tic)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 55 rows, 155 columns and 405 nonzeros
Model fingerprint: 0x42759c4b
Variable types: 150 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-02, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 4e+01]
Presolve removed 55 rows and 155 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 20.3 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.030000000000e+01, best bound 2.030000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 55 rows, 155 columns and 405 nonzeros
Model fingerprint

     0     0 4940.90000    0    1 4957.90000 4940.90000  0.34%     -    0s
H    0     0                    4956.1000000 4940.90000  0.31%     -    0s

Cutting planes:
  Gomory: 2
  Implied bound: 4
  MIR: 1
  Flow cover: 21

Explored 1 nodes (37 simplex iterations) in 0.02 seconds
Thread count was 12 (of 12 available processors)

Solution count 3: 4956.1 4976.7 4994.9 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.956100000000e+03, best bound 4.955896816018e+03, gap 0.0041%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 55 rows, 155 columns and 405 nonzeros
Model fingerprint: 0x02c6b6b9
Variable types: 150 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 4e+01]
Presolve removed 40 rows and 40 columns
Presolve time: 0.00s
Presolve

In [51]:
results

{0.1: (20, [0.0, 0.0, 0.0, 0.0, 0.0]),
 0.5: (89, [1.0, 1.0, -0.0, -0.0, -0.0]),
 1: (115, [1.0, 1.0, -0.0, -0.0, -0.0]),
 10: (560, [1.0, 1.0, -0.0, -0.0, -0.0]),
 20: (1053, [1.0, 1.0, -0.0, -0.0, -0.0]),
 100: (4956, [1.0, 1.0, 0.0, 1.0, -0.0]),
 1000: (48804, [1.0, 1.0, 1.0, -0.0, -0.0])}